In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN','NAME'])
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,Association,1,0,N,5000,1
1,T3,Independent,C2000,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Association,1,0,N,5000,0
34297,T5,Independent,C3000,Association,1,0,N,5000,1


In [3]:
#OPTIMIZATION ATTEMPT 5
bins = [0,5000,10000,25000,50000,75000,100000,500000,1000000,100000000]
labels = ['<5k','5k-10k','10K-25K','25K-50K','50K-75K','75k-100k','100k-500k','500k-1M','>1M']
application_df['Ask_amt'] = pd.cut(application_df['ASK_AMT'], bins, labels = labels)


application_df['Ask_amt'].value_counts()

application_df = application_df.drop(columns=['ASK_AMT'])
    
application_df


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,IS_SUCCESSFUL,Ask_amt
0,T10,Independent,C1000,Association,1,0,N,1,<5k
1,T3,Independent,C2000,Co-operative,1,1-9999,N,1,100k-500k
2,T5,CompanySponsored,C3000,Association,1,0,N,0,<5k
3,T3,CompanySponsored,C2000,Trust,1,10000-24999,N,1,5k-10k
4,T3,Independent,C1000,Trust,1,100000-499999,N,1,100k-500k
...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,Association,1,0,N,0,<5k
34295,T4,CompanySponsored,C3000,Association,1,0,N,0,<5k
34296,T3,CompanySponsored,C2000,Association,1,0,N,0,<5k
34297,T5,Independent,C3000,Association,1,0,N,1,<5k


In [4]:
# Look at APPLICATION_TYPE value counts for binning
app_vc = application_df['APPLICATION_TYPE'].value_counts()
app_vc

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Determine which values to replace if counts are less than ...?
replace_application = list(app_vc[app_vc<500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
class_vc = application_df['CLASSIFICATION'].value_counts()

In [7]:
# Determine which values to replace if counts are less than ..?
replace_class = list(class_vc[class_vc<1880].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [8]:
application_df.dtypes
application_df['Ask_amt'] = application_df['Ask_amt'].astype(object)

In [9]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS',
 'Ask_amt']

In [10]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))


# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

C:\Users\jkova\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,Ask_amt_100k-500k,Ask_amt_10K-25K,Ask_amt_25K-50K,Ask_amt_500k-1M,Ask_amt_50K-75K,Ask_amt_5k-10k,Ask_amt_75k-100k,Ask_amt_<5k,Ask_amt_>1M,Ask_amt_nan
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat, axis=1)
application_df.head()

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,Ask_amt_100k-500k,Ask_amt_10K-25K,Ask_amt_25K-50K,Ask_amt_500k-1M,Ask_amt_50K-75K,Ask_amt_5k-10k,Ask_amt_75k-100k,Ask_amt_<5k,Ask_amt_>1M,Ask_amt_nan
0,1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
print(y)
X = application_df.drop(columns='IS_SUCCESSFUL').values
print(X)
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=89)

[1 1 0 ... 0 1 0]
[[1. 0. 1. ... 1. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 1. 0. 0.]
 ...
 [1. 0. 0. ... 1. 0. 0.]
 [1. 0. 0. ... 1. 0. 0.]
 [1. 0. 0. ... 0. 1. 0.]]


In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [15]:
#OPTIMIZATION ATTEMPT 1, increase neurons in each layer. Acc. score: 0.73
#OPTIMIZATION ATTEMPT 2, add a third hidden layer. Acc. score: 0.73
#OPTIMIZATION ATTEMPT 3, change activation functions. Acc. score: 0.73
#OPTIMIZATION ATTMEPT 4, add epochs. No change
#OPTIMIZATION ATTEMPT #5, tried binning the 'ASK_AMT' column of the dataframe, no change
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nom_input_features = len(X_train_scaled[0])
hidden_layers_node_1 = 8
hidden_layers_node_2 = 24
hidden_layers_node_3 = 24

nn = tf.keras.models.Sequential()

nn.add(
    tf.keras.layers.Dense(units=hidden_layers_node_1, input_dim=nom_input_features, activation="tanh")
)


# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layers_node_2, activation="sigmoid"))

nn.add(tf.keras.layers.Dense(units=hidden_layers_node_2, activation="sigmoid"))

#Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layers_node_3, activation= 'relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 384       
                                                                 
 dense_1 (Dense)             (None, 20)                180       
                                                                 
 dense_2 (Dense)             (None, 20)                420       
                                                                 
 dense_3 (Dense)             (None, 40)                840       
                                                                 
 dense_4 (Dense)             (None, 1)                 41        
                                                                 
Total params: 1,865
Trainable params: 1,865
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=200,callbacks=[cp_callback])

Epoch 1/200
764/804 [===========================>..] - ETA: 0s - loss: 0.6107 - accuracy: 0.6798
Epoch 1: saving model to checkpoints\weights.01.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6096 - accuracy: 0.6808
Epoch 2/200
799/804 [============================>.] - ETA: 0s - loss: 0.5715 - accuracy: 0.7227
Epoch 2: saving model to checkpoints\weights.02.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5711 - accuracy: 0.7230
Epoch 3/200
794/804 [============================>.] - ETA: 0s - loss: 0.5645 - accuracy: 0.7252
Epoch 3: saving model to checkpoints\weights.03.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5650 - accuracy: 0.7247
Epoch 4/200
802/804 [============================>.] - ETA: 0s - loss: 0.5623 - accuracy: 0.7263
Epoch 4: saving model to checkpoints\weights.04.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5624 - accuracy: 0.7263
Epoch 5/200
758/804 [=======================

779/804 [============================>.] - ETA: 0s - loss: 0.5497 - accuracy: 0.7309
Epoch 35: saving model to checkpoints\weights.35.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5504 - accuracy: 0.7301
Epoch 36/200
799/804 [============================>.] - ETA: 0s - loss: 0.5505 - accuracy: 0.7317
Epoch 36: saving model to checkpoints\weights.36.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5503 - accuracy: 0.7317
Epoch 37/200
802/804 [============================>.] - ETA: 0s - loss: 0.5497 - accuracy: 0.7321
Epoch 37: saving model to checkpoints\weights.37.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5495 - accuracy: 0.7322
Epoch 38/200
771/804 [===========================>..] - ETA: 0s - loss: 0.5493 - accuracy: 0.7318
Epoch 38: saving model to checkpoints\weights.38.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5498 - accuracy: 0.7314
Epoch 39/200
769/804 [===========================

771/804 [===========================>..] - ETA: 0s - loss: 0.5490 - accuracy: 0.7315
Epoch 69: saving model to checkpoints\weights.69.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5477 - accuracy: 0.7322
Epoch 70/200
777/804 [===========================>..] - ETA: 0s - loss: 0.5476 - accuracy: 0.7340
Epoch 70: saving model to checkpoints\weights.70.hdf5
804/804 [==============================] - 1s 996us/step - loss: 0.5477 - accuracy: 0.7336
Epoch 71/200
782/804 [============================>.] - ETA: 0s - loss: 0.5483 - accuracy: 0.7326
Epoch 71: saving model to checkpoints\weights.71.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5478 - accuracy: 0.7331
Epoch 72/200
804/804 [==============================] - ETA: 0s - loss: 0.5480 - accuracy: 0.7331
Epoch 72: saving model to checkpoints\weights.72.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5480 - accuracy: 0.7331
Epoch 73/200
758/804 [=========================

764/804 [===========================>..] - ETA: 0s - loss: 0.5467 - accuracy: 0.7333
Epoch 103: saving model to checkpoints\weights.103.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5465 - accuracy: 0.7329
Epoch 104/200
758/804 [===========================>..] - ETA: 0s - loss: 0.5468 - accuracy: 0.7328
Epoch 104: saving model to checkpoints\weights.104.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5468 - accuracy: 0.7329
Epoch 105/200
782/804 [============================>.] - ETA: 0s - loss: 0.5472 - accuracy: 0.7330
Epoch 105: saving model to checkpoints\weights.105.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5467 - accuracy: 0.7334
Epoch 106/200
765/804 [===========================>..] - ETA: 0s - loss: 0.5452 - accuracy: 0.7339
Epoch 106: saving model to checkpoints\weights.106.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5466 - accuracy: 0.7326
Epoch 107/200
799/804 [===============

804/804 [==============================] - 1s 1ms/step - loss: 0.5459 - accuracy: 0.7340
Epoch 137/200
775/804 [===========================>..] - ETA: 0s - loss: 0.5462 - accuracy: 0.7337
Epoch 137: saving model to checkpoints\weights.137.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5460 - accuracy: 0.7338
Epoch 138/200
783/804 [============================>.] - ETA: 0s - loss: 0.5462 - accuracy: 0.7330
Epoch 138: saving model to checkpoints\weights.138.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5456 - accuracy: 0.7334
Epoch 139/200
797/804 [============================>.] - ETA: 0s - loss: 0.5455 - accuracy: 0.7340
Epoch 139: saving model to checkpoints\weights.139.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5460 - accuracy: 0.7334
Epoch 140/200
783/804 [============================>.] - ETA: 0s - loss: 0.5466 - accuracy: 0.7337
Epoch 140: saving model to checkpoints\weights.140.hdf5
804/804 [===============

763/804 [===========================>..] - ETA: 0s - loss: 0.5452 - accuracy: 0.7343
Epoch 170: saving model to checkpoints\weights.170.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5448 - accuracy: 0.7344
Epoch 171/200
795/804 [============================>.] - ETA: 0s - loss: 0.5451 - accuracy: 0.7339
Epoch 171: saving model to checkpoints\weights.171.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5451 - accuracy: 0.7338
Epoch 172/200
773/804 [===========================>..] - ETA: 0s - loss: 0.5460 - accuracy: 0.7333
Epoch 172: saving model to checkpoints\weights.172.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5453 - accuracy: 0.7338
Epoch 173/200
773/804 [===========================>..] - ETA: 0s - loss: 0.5459 - accuracy: 0.7341
Epoch 173: saving model to checkpoints\weights.173.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5450 - accuracy: 0.7349
Epoch 174/200
802/804 [===============